In [1]:
import pkg_resources

ls_libs = ["numpy", "pandas", "scipy", "scikit-learn"]
for n_lib in ls_libs:
    vak_ver = pkg_resources.get_distribution(n_lib).version
    print(f"{n_lib}: {vak_ver}")

numpy: 2.2.3
pandas: 2.2.3
scipy: 1.15.2
scikit-learn: 1.6.1


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

### Q1.

In [22]:
df = pd.read_csv("set_02_data.csv")
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,6682-VCIXC,Female,0,Yes,Yes,43,No,No phone service,DSL,Yes,...,No,Yes,Yes,No,One year,Yes,Bank transfer (automatic),51.25,2151.6,No
1,2486-WYVVE,Male,0,Yes,No,31,Yes,Yes,DSL,No,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,79.30,2484.0,No


In [31]:
df_q1 = df.loc[(df["gender"] == "Male") & (df["SeniorCitizen"] == 0), "OnlineSecurity":"StreamingMovies"].copy()
df_q1.head(1)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
1,No,No,Yes,Yes,Yes,Yes


In [ ]:
# df_q1["OnlineSecurity"].unique()
df_q1["OnlineBackup"].unique()

In [ ]:
# df_q1.unique() # DataFrame에는 .unique() 메서드가 없음
# df_q1.drop_duplicates()
# df_q1.apply(lambda x: [x.unique()]) # 시험버전, 암기 할 필요 없음
df_q1.apply(lambda x: x.unique()) # 최신버전, 암기 할 필요 없음

In [34]:
df_q1_sub = df_q1.loc[df_q1["OnlineSecurity"].isin(["Yes", "No"]), ]
# df_q1_sub = df_q1_sub.loc[df_q1["OnlineBackup"].isin(["Yes", "No"]), ]

In [ ]:
df_q1_sub.apply(lambda x: x.unique())

In [ ]:
df_q1_sub.head()

In [ ]:
ser_cnt = df_q1_sub.apply(lambda x: sum(x == "Yes"), axis = 1)
ser_cnt2 = ser_cnt.value_counts()
ser_cnt2

In [38]:
ser_cnt2[1] / ser_cnt2[6]

np.float64(3.288135593220339)

In [39]:
np.floor(ser_cnt2[1] / ser_cnt2[6] * 100) / 100

np.float64(3.28)

### Q2.

In [ ]:
df = pd.read_csv("set_02_data.csv")
df.head(1)

In [ ]:
df["PaymentMethod"].unique()

In [48]:
df_A = df.loc[ df["PaymentMethod"].str.contains("automatic"), ["tenure", "MonthlyCharges", "TotalCharges"]]
df_B = df.loc[~df["PaymentMethod"].str.contains("automatic"), ["tenure", "MonthlyCharges", "TotalCharges"]]
len(df), len(df_A), len(df_B)

(6465, 2805, 3660)

In [49]:
df_A["dur"] = df_A["TotalCharges"] // df_A["MonthlyCharges"]
df_B["dur"] = df_B["TotalCharges"] // df_B["MonthlyCharges"]

In [59]:
# df_A.drop(columns = "TotalCharges").corr() # .corr() 은 기본적으로 피어슨 상관분석으로 동작
df_A_corr = df_A.iloc[:, [0, 1, 3]].corr()
val_A_corr = df_A_corr.replace(1, np.nan).max().max()
val_A_corr

np.float64(0.9985017739420756)

In [60]:
df_B_corr = df_B.iloc[:, [0, 1, 3]].corr()
val_B_corr = df_B_corr.replace(1, np.nan).max().max()
val_B_corr

np.float64(0.9989839520673992)

In [62]:
round(abs(val_A_corr - val_B_corr), 4)

np.float64(0.0005)

### Q3.

In [122]:
df = pd.read_csv("set_02_data.csv")
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,6682-VCIXC,Female,0,Yes,Yes,43,No,No phone service,DSL,Yes,...,No,Yes,Yes,No,One year,Yes,Bank transfer (automatic),51.25,2151.6,No
1,2486-WYVVE,Male,0,Yes,No,31,Yes,Yes,DSL,No,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,79.30,2484.0,No


In [123]:
ls_x1 = ["SeniorCitizen", "Partner", "Dependents", "tenure", "MonthlyCharges", "TotalCharges"]
ls_x2 = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingMovies", "PaperlessBilling"]
df_q3 = df[["customerID", "Churn"] + ls_x1 + ls_x2].copy()

In [124]:
df_q3 = df_q3.replace({"Yes": 1, "No": 0})

C:\Users\PC\AppData\Local\Temp\ipykernel_25588\3322953265.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_q3 = df_q3.replace({"Yes": 1, "No": 0})


In [125]:
df_q3.head(2)

,customerID,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
0,6682-VCIXC,0,0,1,1,43,51.25,2151.6,1,1,0,1,0,1
1,2486-WYVVE,0,0,1,0,31,79.30,2484.0,0,0,1,1,1,1


In [126]:
# df_q3_obj = df_q3.select_dtypes(include = "object")
df_q3_obj = df_q3.loc[:, df_q3.dtypes == "object"] # 시험버전
df_q3_obj.head(2)

,customerID,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies
0,6682-VCIXC,1,1,0,1,0
1,2486-WYVVE,0,0,1,1,1


In [ ]:
df_q3_obj.iloc[:, 1:].apply(lambda x: x.unique())

In [ ]:
df_q3 = df_q3.replace({"Pending": -1, "No internet service": -1, "Under despute": -1})

In [88]:
df_q3_id = df_q3["customerID"].str.split("-", expand = True)
df_q3_id.head(2)

,0,1
0,6682,VCIXC
1,2486,WYVVE


In [114]:
df_train = df_q3.loc[(df_q3_id[0].astype("int") % 4) != 0, ].reset_index(drop = True)
df_test  = df_q3.loc[(df_q3_id[0].astype("int") % 4) == 0, ].reset_index(drop = True)

In [96]:
len(df_train), len(df_test)

(4832, 1633)

In [98]:
df_train_num = df_train.loc[:, "SeniorCitizen":"TotalCharges"]
df_test_num = df_test.loc[:, "SeniorCitizen":"TotalCharges"]

In [101]:
model_nor = MinMaxScaler().fit(df_train_num)
arr_train_num_nor = model_nor.transform(df_train_num)
arr_test_num_nor = model_nor.transform(df_test_num)

In [106]:
df_train_num_nor = pd.DataFrame(arr_train_num_nor, columns = df_train_num.columns)
df_test_num_nor  = pd.DataFrame(arr_test_num_nor, columns = df_test_num.columns)

In [116]:
df_train_X = pd.concat([df_train_num_nor, df_train.loc[:, "OnlineSecurity":]], axis = 1)
df_test_X  = pd.concat([df_test_num_nor,  df_test.loc[:,  "OnlineSecurity":]], axis = 1)

In [119]:
model_lr = LogisticRegression(random_state = 123)
model_lr.fit(X = df_train_X, 
             y = df_train["Churn"])
pred = model_lr.predict(df_test_X)

In [121]:
round(f1_score(y_true = df_test["Churn"], y_pred = pred), 2)

0.58